In [1]:
# I need to run models using this data...
# Fantasy points - what we're trying to predict
# I want to predict fantasy points 1, 2, and 3 years out if possible
# I'll need to join the seasonal stats to get target share, usage, fantasy points to predict
# Actually..... I need to grab the fantasy points and then rank them either with an integer or with min-max normalization to see the percentages
# Then I need combine data using shit like bench, 40 dash, 3-cone drill, etc
# Build the query first and then we can do the next step

# I think the purpose of this is to identify the best traits and then do a ranking of the 2024 prospects to
# identify any sleepers
# Might need to use some college stats as well so that the uber athletes are getting boosted too high

In [9]:
# Import packages

import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

In [254]:
# Configure SQL

def execute_statement(sql: str):
    with psycopg2.connect(host="localhost", database="thefantasybot", user="tbakely") as conn:
         df = pd.read_sql(sql, conn)
         return df
    
model_dataset = """
WITH AggSnaps AS (
SELECT
	  id
	, season
	, SUM(offense_snaps) as offense_snaps
	, AVG(offense_pct) as offense_pct
 FROM archive_data.offense_snap_counts
GROUP BY id, season
),
AggRedzone AS (
SELECT
	  player_id
	, season
	, SUM(redzone) as redzone
	, AVG(redzone) as avg_redzone
 FROM archive_data.redzone_snaps
GROUP BY player_id, season
),
RookieTable AS (
SELECT
	  name
	, gsis_id
	, a.season
	, b.position
	, targets AS rookie_targets
	, tgt_sh AS rookie_tgt_sh
	, offense_snaps AS rookie_offense_snaps
	, offense_pct AS rookie_offense_pct
	, rushing_epa AS rookie_rushing_epa
	, receiving_epa AS rookie_receiving_epa
	, rushing_epa + receiving_epa AS rookie_total_epa
	, rushing_first_downs + receiving_first_downs AS rookie_total_first_downs
	, fantasy_points AS rookie_standard_points
	, fantasy_points_ppr AS rookie_ppr_points
	, redzone AS rookie_redzone
	, avg_redzone AS rookie_avg_redzone
	, carries AS rookie_carries
	, carries + targets AS rookie_total_usage
 FROM archive_data.seasonal_data a
INNER JOIN archive_data.full_ids b
   ON a.player_id = b.gsis_id
 LEFT JOIN AggSnaps c
   ON a.player_id = c.id
  AND a.season = c.season
 LEFT JOIN AggRedzone d
   ON a.player_id = d.player_id
  AND a.season = d.season
WHERE a.season = b.draft_year
  AND a.season = c.season
  AND b.position IN ('RB')
)
SELECT
	  player_name
	, b.gsis_id
	, a.pfr_id
	, age
	, a.draft_year
	, a.draft_round
	, b.draft_pick
	, a.draft_ovr
	, ht
	, height
	, wt
	, weight
	, school
	, db_season
	, forty
	, bench
	, vertical
	, broad_jump
	, cone
	, rookie_targets
	, rookie_tgt_sh
	, rookie_offense_snaps
	, rookie_offense_pct
	, rookie_rushing_epa
	, rookie_receiving_epa
	, rookie_total_epa
	, rookie_total_first_downs
	, rookie_standard_points
	, rookie_ppr_points
	, rookie_redzone
	, rookie_avg_redzone
	, rookie_carries
	, rookie_total_usage
 FROM archive_data.combine_data a
RIGHT JOIN archive_data.full_ids b
   ON a.pfr_id = b.pfr_id
RIGHT JOIN RookieTable c
   ON b.gsis_id = c.gsis_id
"""

model_data = execute_statement(model_dataset)

response_vars = [
	"rookie_targets",
	"rookie_tgt_sh",
	"rookie_offense_snaps",
	"rookie_offense_pct",
	"rookie_rushing_epa",
	"rookie_receiving_epa",
	"rookie_total_epa",
	"rookie_total_first_downs",
	"rookie_standard_points",
	"rookie_ppr_points",
	"rookie_redzone",
	"rookie_avg_redzone",
	"rookie_carries",
	"rookie_total_usage",
]

predictors = [
     # "age",
     # "draft_round",
     "draft_ovr",
     "height",
     "weight",
     # "school",
     "forty",
     "bench",
     "vertical",
     "broad_jump",
     "cone",
]

In [255]:
# Fill null values with 9999 for draft rounds and forward fill on the school

model_data1 = model_data[predictors + response_vars] 
for col in ["draft_ovr"]:
    model_data1[col] = model_data1[col].fillna(9999)

In [256]:
nulls = model_data1.isnull().mean().sort_values(ascending=False)
nulls[nulls > 0]

# we dont need player_name, pfr_id, gsis_id, draft_pick, or draft year really right?
# draft_round to 9999, draft_ovr to 9999
# dont need ht, wt

model_no_nulls = model_data1.dropna()

In [79]:
model_data1 = model_data1.sample(frac=1).reset_index(drop=True)
model_data1["school"] = model_data1["school"].fillna(method="ffill") # Can this step be done in a pipeline I wonder?

In [115]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder # this is only applied to the y column
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

column_data = []
for response in response_vars:

    # rookie_targets
    X = model_data1[predictors]
    y = KNNImputer().fit_transform(model_data1[[response]])

    # training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

    # Build pipeline data

    # encoder = OneHotEncoder(sparse=False)
    # transformer = ColumnTransformer(
    #     transformers=[
    #         ("encoder", encoder, ["school"])
    #     ]
    # )

    imputer = KNNImputer()
    rfr = RandomForestRegressor(n_estimators=1000)
    pipe = Pipeline([
        #("transformer", transformer),
        ("imputer", imputer),
        ("rfr", rfr)
    ])

    # Fitting the model
    pipe.fit(X, y)

    # y_pred = pipe.predict(X_test)

    forest = pipe.named_steps["rfr"]
    column = forest.feature_importances_
    column_data.append(column)


importances_table = pd.DataFrame(dict(zip(response_vars, column_data)), index=predictors)

In [116]:
importances_table

,rookie_targets,rookie_tgt_sh,rookie_offense_snaps,rookie_offense_pct,rookie_rushing_epa,rookie_receiving_epa,rookie_total_epa,rookie_total_first_downs,rookie_standard_points,rookie_ppr_points,rookie_redzone,rookie_avg_redzone,rookie_carries,rookie_total_usage
age,0.121730,0.115007,0.134419,0.123398,0.130908,0.166086,0.160952,0.116935,0.122048,0.121082,0.120202,0.109950,0.089198,0.100908
draft_round,0.014256,0.011212,0.008304,0.007562,0.019200,0.012080,0.013332,0.011307,0.010525,0.010342,0.012649,0.025367,0.017123,0.011262
draft_ovr,0.400440,0.323193,0.400626,0.371515,0.127339,0.187809,0.168501,0.340834,0.350934,0.384130,0.192943,0.128275,0.201090,0.274326
height,0.053620,0.075694,0.057144,0.067645,0.115708,0.073143,0.093932,0.081585,0.076445,0.065815,0.131134,0.137447,0.160798,0.128186
weight,0.075019,0.091097,0.074868,0.080727,0.119515,0.103769,0.117094,0.111183,0.101445,0.080415,0.187060,0.183209,0.221827,0.163949
forty,0.071305,0.076283,0.079786,0.080485,0.103810,0.093795,0.091658,0.090240,0.084389,0.084583,0.077999,0.089591,0.064520,0.078032
bench,0.075004,0.094479,0.061465,0.071088,0.100169,0.093294,0.091229,0.056314,0.062415,0.057434,0.069128,0.081931,0.071270,0.062727
vertical,0.059120,0.062767,0.054431,0.061190,0.100674,0.087679,0.087431,0.061786,0.059170,0.060407,0.068214,0.075837,0.049723,0.059823
broad_jump,0.058511,0.071450,0.065944,0.070116,0.087321,0.085975,0.083236,0.063126,0.061702,0.061406,0.062163,0.073325,0.056739,0.056617
cone,0.070995,0.078817,0.063014,0.066273,0.095356,0.096370,0.092635,0.066688,0.070927,0.074386,0.078506,0.095070,0.067711,0.064171


In [106]:
# Fitting the model
pipe.fit(X, y)

y_pred = pipe.predict(X_test)
r2_score(y_test, y_pred)

0.9047963542936686

In [132]:
# Find best params for the random forsest regressor
from sklearn.model_selection import GridSearchCV

# rookie_targets
X = KNNImputer().fit_transform(model_data1[predictors].values)
y = KNNImputer().fit_transform(model_data1[["rookie_tgt_sh"]].values)

# training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Randoom Forest regressor
rfc_params = {
    "n_estimators": [10, 50, 100, 500, 1000, 5000],
    "max_depth": [None, 5, 10, 100, 1000],
    "max_features": [None, "sqrt"]
}
grid = GridSearchCV(RandomForestRegressor(), rfc_params, cv=5)
grid.fit(X, y)


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 5, 10, 100, 1000],
                         'max_features': [None, 'sqrt'],
                         'n_estimators': [10, 50, 100, 500, 1000, 5000]})

In [149]:
grid.best_score_

0.2807749024985112

In [243]:
# rookie_targets
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = model_no_nulls[predictors].values
y = model_no_nulls[["rookie_avg_redzone"]].values

# X = KNNImputer().fit_transform(model_data1[predictors].values)
# y = KNNImputer().fit_transform(model_data1[["rookie_avg_redzone"]].values)
y_new = np.where(y >= 1.74, 1, 0) # WR/TE scoring threshold

# training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size=0.3, random_state=0)

# RFR based on the grid search
# rfr = RandomForestRegressor(n_estimators=50, max_depth=None)
rfr = RandomForestClassifier(n_estimators=5000, max_depth=5)
rfr.fit(X_train, y_train)

# Predict results
y_pred = rfr.predict(X_test)
accuracy_score(y_test, y_pred)

0.7560975609756098

In [241]:
y_new[y_new==1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1])

In [244]:
pd.DataFrame({"feature_importance":rfr.feature_importances_}, index=predictors)

,feature_importance
age,0.111963
draft_ovr,0.112916
height,0.050723
weight,0.070168
forty,0.072733
bench,0.078902
vertical,0.145408
broad_jump,0.148479
cone,0.208708


In [270]:
# rookie_targets
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = model_no_nulls[predictors].values
y = model_no_nulls[["rookie_redzone"]].values

y_new = np.where(y >= 25, 1, 0) # RB scoring threshold

# training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y_new, test_size=0.3, random_state=0)

# RFR based on the grid search
rfr = RandomForestClassifier(n_estimators=5000, max_depth=None)
rfr.fit(X_train, y_train)

# Predict results
y_pred = rfr.predict(X_test)
accuracy_score(y_test, y_pred)

0.88

In [271]:
y_new[y_new==1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [272]:
pd.DataFrame({"feature_importance":rfr.feature_importances_}, index=predictors)

,feature_importance
draft_ovr,0.247521
height,0.065401
weight,0.109133
forty,0.108809
bench,0.110684
vertical,0.111517
broad_jump,0.103267
cone,0.143668


In [266]:
model_no_nulls.sort_values("rookie_redzone", ascending=False).head(20)

,draft_ovr,height,weight,forty,bench,vertical,broad_jump,cone,rookie_targets,rookie_tgt_sh,rookie_offense_snaps,rookie_offense_pct,rookie_rushing_epa,rookie_receiving_epa,rookie_total_epa,rookie_total_first_downs,rookie_standard_points,rookie_ppr_points,rookie_redzone,rookie_avg_redzone,rookie_carries,rookie_total_usage
177,48.0,73.0,225.0,4.60,24.0,31.5,118.0,6.75,66,0.138947,677.0,0.775385,-25.268671,11.073727,-14.194944,61.0,171.9,216.9,59.0,4.538462,244,310
71,41.0,70.0,226.0,4.39,17.0,36.0,123.0,7.01,39,0.076772,558.0,0.512500,14.960917,13.237644,28.198561,81.0,216.8,252.8,58.0,3.625000,232,271
200,31.0,69.0,210.0,4.46,28.0,36.0,120.0,6.79,70,0.123675,821.0,0.782500,-4.398772,12.980267,8.581495,76.0,262.6,311.6,56.0,4.000000,319,389
202,173.0,70.0,222.0,4.63,16.0,35.5,117.0,7.01,16,0.036199,763.0,0.712353,-1.299136,-0.879558,-2.178694,86.0,241.0,252.0,55.0,3.235294,335,351
191,160.0,69.0,224.0,4.55,27.0,33.0,122.0,6.70,35,0.096953,566.0,0.672857,-26.000682,-0.007196,-26.007877,54.0,157.4,183.4,43.0,3.583333,250,285
84,53.0,71.0,210.0,4.49,20.0,36.0,124.0,6.89,63,0.102773,660.0,0.537059,-14.837368,18.447417,3.610049,49.0,168.7,218.7,38.0,2.533333,179,242
54,35.0,70.0,220.0,4.55,22.0,36.0,123.0,6.97,53,0.097967,551.0,0.507059,-17.262295,7.522279,-9.740017,64.0,161.9,204.9,36.0,2.400000,203,256
194,170.0,70.0,224.0,4.59,23.0,33.0,115.0,7.03,27,0.042994,651.0,0.522941,-16.885638,3.392696,-13.492942,57.0,114.6,131.6,32.0,2.461538,211,238
143,125.0,72.0,218.0,4.53,11.0,35.5,121.0,6.96,62,0.091716,393.0,0.349375,-28.276297,-14.876773,-43.153070,44.0,100.7,145.7,30.0,2.307692,137,199
91,35.0,71.0,227.0,4.52,29.0,38.5,128.0,7.09,29,0.050523,395.0,0.387500,1.853204,2.026853,3.880056,52.0,174.5,194.5,30.0,2.727273,192,221


In [107]:
forest = pipe.named_steps["rfr"]
forest.feature_importances_
pd.DataFrame({"feature_importances": forest.feature_importances_}, index=predictors).sort_values("feature_importances", ascending=False)

,feature_importances
draft_ovr,0.403170
age,0.120881
weight,0.074317
bench,0.074236
cone,0.072498
forty,0.071412
broad_jump,0.057902
vertical,0.057411
height,0.054534
draft_round,0.013640


In [45]:
model_data1.head(20)

# Draft year nan seems to be UDFAs


,age,draft_round,draft_ovr,height,weight,school,forty,bench,vertical,broad_jump,cone,rookie_targets,rookie_tgt_sh,rookie_offense_snaps,rookie_offense_pct,rookie_rushing_epa,rookie_receiving_epa,rookie_total_epa,rookie_total_first_downs,rookie_standard_points,rookie_ppr_points,rookie_redzone,rookie_avg_redzone,rookie_carries,rookie_total_usage
0,21.3,5.0,143.0,71.0,215.0,Pittsburgh,NaN,NaN,NaN,NaN,NaN,11,0.050691,77.0,0.191667,-2.945533,-1.215488,-4.161021,5.0,11.3,18.3,NaN,NaN,22,33
1,22.3,3.0,84.0,69.0,188.0,Texas A&M,4.32,NaN,33.0,NaN,NaN,37,0.106322,287.0,0.462000,33.233975,1.491719,34.725694,40.0,163.7,190.7,25.0,3.571429,103,140
2,22.0,1.0,23.0,71.0,175.0,USC,4.49,NaN,34.0,122.0,NaN,108,0.171157,921.0,0.810588,-0.410607,26.098682,25.688076,38.0,151.3,221.3,16.0,1.600000,1,109
3,23.0,5.0,175.0,78.0,245.0,Clemson,NaN,NaN,38.5,125.0,NaN,11,0.083969,211.0,0.190625,0.000000,9.994334,9.994334,7.0,15.5,25.5,2.0,1.000000,0,11
4,24.0,7.0,253.0,72.0,190.0,Michigan,4.54,14.0,38.5,120.0,6.98,13,0.054393,183.0,0.165882,0.000000,6.735426,6.735426,5.0,24.8,30.8,3.0,1.000000,0,13
5,22.4,3.0,88.0,72.0,213.0,Auburn,4.56,NaN,32.5,119.0,NaN,4,0.008989,141.0,0.122941,-12.336057,-13.688695,-26.024752,12.0,23.8,24.8,9.0,1.500000,50,54
6,25.5,9999.0,9999.0,76.0,207.0,UCLA,NaN,NaN,NaN,NaN,NaN,25,0.051020,303.0,0.284118,0.775604,8.033546,8.809151,10.0,37.9,56.9,5.0,1.666667,1,26
7,21.7,6.0,187.0,72.0,197.0,LSU,4.50,NaN,29.0,118.0,NaN,7,0.060345,116.0,0.330000,0.000000,-2.691037,-2.691037,1.0,1.9,3.9,1.0,1.000000,0,7
8,23.7,7.0,244.0,74.0,205.0,South Carolina,4.69,NaN,35.0,130.0,7.15,6,0.051724,77.0,0.148571,0.000000,3.854871,3.854871,4.0,6.4,12.4,NaN,NaN,0,6
9,23.9,5.0,163.0,69.0,215.0,Illinois,4.43,NaN,40.0,127.0,NaN,15,0.041551,93.0,0.118333,-9.011016,6.435282,-2.575734,10.0,39.5,53.5,5.0,1.250000,44,59
